PyTorch 提供了两个用于 处理数据的原语: torch.utils.data.DataLoader 和 torch.utils.data.Dataset。`Dataset`存储样本及其对应的标签，而`DataLoader`则在`Dataset`外部封装一层，变为可迭代对象。

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

将 Dataset 作为参数传递给 DataLoader，在数据集上封装了一个可迭代对象， 支持自动批处理、采样、打乱和多进程数据加载。这里我们定义一个批处理大小为 64， 即 dataloader 每批将返回大小为 64 的特征数据和标签。

In [6]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


X 的形状中 N = 64 即为样本数为 64，C = 1 表示每个样本有一个通道（灰度图），H 和 W 分别为长和宽。 y 的形状表示有 64 个对应的标签，且标签数据类型为 int，为了对应数字。

In [7]:
print(torch.cuda.is_available())

True


In [8]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


训练一个模型需要一个损失函数和一个优化器

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

检查模型在测试数据集上的效果，以确保它在持续学习。

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100*(correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.310095  [   64/60000]
loss: 2.298122  [ 6464/60000]
loss: 2.279445  [12864/60000]
loss: 2.268215  [19264/60000]
loss: 2.255812  [25664/60000]
loss: 2.216004  [32064/60000]
loss: 2.228203  [38464/60000]
loss: 2.189994  [44864/60000]
loss: 2.184163  [51264/60000]
loss: 2.151459  [57664/60000]
Test Error: 
 Accuracy: 39.0%, Avg loss: 2.151487

Epoch 2
-------------------------------
loss: 2.166918  [   64/60000]
loss: 2.155850  [ 6464/60000]
loss: 2.096817  [12864/60000]
loss: 2.108942  [19264/60000]
loss: 2.070046  [25664/60000]
loss: 1.992161  [32064/60000]
loss: 2.021636  [38464/60000]
loss: 1.940024  [44864/60000]
loss: 1.942545  [51264/60000]
loss: 1.866510  [57664/60000]
Test Error: 
 Accuracy: 55.6%, Avg loss: 1.873620

Epoch 3
-------------------------------
loss: 1.910398  [   64/60000]
loss: 1.878396  [ 6464/60000]
loss: 1.761187  [12864/60000]
loss: 1.799127  [19264/60000]
loss: 1.711958  [25664/60000]
loss: 1.642900  [32064/60000

In [13]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [14]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [15]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
